# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Read in csv file from WeatherPy
cities = pd.read_csv("../output_data/CityWeatherData.csv", encoding="utf-8")
cities_df = pd.DataFrame(cities)
#Display
cities_df.head()

,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temp (F),Windspeed (MPH),Latitude,Longitude
0,atuona,3,PF,1970-01-01 00:00:01.603342249,74,-9.80,-139.03,77.92,15.55,-9.80,-139.03
1,ndele,94,CF,1970-01-01 00:00:01.603341837,81,8.41,20.65,70.20,2.89,8.41,20.65
2,dikson,99,RU,1970-01-01 00:00:01.603342265,96,73.51,80.55,25.50,11.99,73.51,80.55
3,new norfolk,75,AU,1970-01-01 00:00:01.603342265,59,-42.78,147.06,61.00,9.17,-42.78,147.06
4,bethel,90,US,1970-01-01 00:00:01.603342265,100,41.37,-73.41,64.99,2.95,41.37,-73.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)
humidity = cities_df['Humidity (%)'].astype(float)
max_humidity = cities_df['Humidity (%)'].max()
max_humidity = max_humidity.astype(float)
locations = cities_df[['Lat', 'Lng']]

In [10]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, 
                                 max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:

narrowed_city_df = cities.loc[(cities["Max Temp (F)"] > 70) & (cities["Max Temp (F)"] < 80) & (cities["Cloudiness (%)"] == 0) & (cities["Windspeed (MPH)"] < 10), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temp (F),Windspeed (MPH),Latitude,Longitude
0,morondava,0,MG,1970-01-01 00:00:01.603342305,72,-20.28,44.28,79.50,9.93,-20.28,44.28
1,jijiga,0,ET,1970-01-01 00:00:01.603341879,22,9.35,42.80,72.01,3.33,9.35,42.80
2,suez,0,EG,1970-01-01 00:00:01.603341882,79,29.97,32.53,71.92,5.93,29.97,32.53
3,huangpi,0,CN,1970-01-01 00:00:01.603341886,60,30.88,114.38,71.60,4.47,30.88,114.38
4,abu dhabi,0,AE,1970-01-01 00:00:01.603342171,65,24.47,54.37,78.80,1.12,24.47,54.37


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temp (F),Windspeed (MPH),Latitude,Longitude,Hotel Name
0,morondava,0,MG,1970-01-01 00:00:01.603342305,72,-20.28,44.28,79.50,9.93,-20.28,44.28,Morondava
1,jijiga,0,ET,1970-01-01 00:00:01.603341879,22,9.35,42.80,72.01,3.33,9.35,42.80,Jijiga
2,suez,0,EG,1970-01-01 00:00:01.603341882,79,29.97,32.53,71.92,5.93,29.97,32.53,Suez
3,huangpi,0,CN,1970-01-01 00:00:01.603341886,60,30.88,114.38,71.60,4.47,30.88,114.38,Wuhan
4,abu dhabi,0,AE,1970-01-01 00:00:01.603342171,65,24.47,54.37,78.80,1.12,24.47,54.37,Abu Dhabi


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))